In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);


make: `/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6' is up to date.


Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                  Mean           SD        Naive SE        MCSE         ESS    
         lp__ -39.50143608  3.2114476528 0.0507774458 0.12402756859  670.448209
accept_stat__   0.94209235  0.0814037101 0.0012871057 0.00118828306 1000.000000
   stepsize__   0.04782780  0.0034161897 0.0000540147 0.00054695977   39.009752
  treedepth__   5.91600000  0.4555171500 0.0072023585 0.01367104370 1000.000000
 n_leapfrog__  73.93600000 34.4533984269 0.5447560608 0.89583057006 1000.000000
  divergent__   0.00000000  0.0000000000 0.0000000000 0.00000000000         NaN
     energy__  45.98944417  4.0704350479 0.0643592291 0.14652291647  771.739180
        alpha   1.07075488  0.7663598392 0.0121172130 0.02646731154  838.389473
  a_society.1  -0.19154250  0.2442448955 0.0038618509 0.00608715953 1000.000000
  a_society.2   0.05176510  0.2184566272 0.0034541026 0.00616129607 1000.000000
  a_

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*